In [ ]:
try:
    import speech_recognition as sr
except:
    ! pip install SpeechRecognition

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 32.8 MB 1.5 MB/s 


In [ ]:
import itertools
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pylab
import speech_recognition as sr
import tensorflow as tf
import wave

from pathlib import Path
from scipy import signal
from scipy.io import wavfile
from sklearn.metrics import confusion_matrix

In [ ]:
directory = "/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/"
INPUT_DIR = directory + "kaggle_data/free-spoken-digit-dataset-master/recordings/"
OUTPUT_DIR = directory + "kaggle_data/preprocessed/"
folder = INPUT_DIR + "0_jackson_0.wav"

In [ ]:
r = sr.Recognizer()

with sr.AudioFile(folder) as source:
    # listen for the data (load audio to memory)
    audio_data = r.record(source)
    # recognize (convert from speech to text)
    text = r.recognize_google(audio_data)
    print(text)

0


In [ ]:
r = sr.Recognizer()
c = 0
for filename in os.listdir(INPUT_DIR):
    
    f = os.path.join(INPUT_DIR, filename)

    with sr.AudioFile(f) as source:
        try:
            audio_data = r.record(source)
            text = r.recognize_google(audio_data)
            print(text)
        
        except:
            continue
    
    print(f)
    if c == 9:
        break
    c += 1

3
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_7.wav
3
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_36.wav
free
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_35.wav
free
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_31.wav
3
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_25.wav
free
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-digit-dataset-master/recordings/3_lucas_22.wav
2
/content/drive/MyDrive/Classes/CSCE 5222 Feature Engineering/Group Project/kaggle_data/free-spoken-dig

In [ ]:
parent_list = os.listdir(INPUT_DIR)
for i in range(10):
    print(parent_list[i])

3_george_3.wav
3_george_30.wav
3_george_29.wav
3_george_28.wav
3_george_31.wav
3_george_27.wav
3_george_26.wav
3_george_24.wav
3_george_23.wav
3_george_25.wav


In [ ]:
# Utility function to get sound and frame rate info
def get_wav_info(wav_file):
    wav = wave.open(wav_file, 'r')
    frames = wav.readframes(-1)
    sound_info = pylab.frombuffer(frames, 'int16')
    frame_rate = wav.getframerate()
    wav.close()
    return sound_info, frame_rate

# For every recording, make a spectogram and save it as label_speaker_no.png
if not os.path.exists(os.path.join(OUTPUT_DIR, 'audio-images')):
    os.mkdir(os.path.join(OUTPUT_DIR, 'audio-images'))
    
l = str(len(os.listdir(INPUT_DIR)))
for i, filename in enumerate(os.listdir(INPUT_DIR)):
    if i % 100 == 0:
        print(str(i) + "/" + l)
    if "wav" in filename:
        file_path = os.path.join(INPUT_DIR, filename)
        file_stem = Path(file_path).stem
        target_dir = f'class_{file_stem[0]}'
        dist_dir = os.path.join(os.path.join(OUTPUT_DIR, 'audio-images'), target_dir)
        file_dist_path = os.path.join(dist_dir, file_stem)
        if not os.path.exists(file_dist_path + '.png'):
            if not os.path.exists(dist_dir):
                os.mkdir(dist_dir)
            file_stem = Path(file_path).stem
            sound_info, frame_rate = get_wav_info(file_path)
            pylab.specgram(sound_info, Fs=frame_rate)
            pylab.savefig(f'{file_dist_path}.png')
            pylab.close()

# Print the ten classes in our dataset
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images'))
print("Classes: \n")
for i in range(10):
    print(path_list[i])
    
# File names for class 1
path_list = os.listdir(os.path.join(OUTPUT_DIR, 'audio-images/class_1'))
print("\nA few example files: \n")
for i in range(10):
    print(path_list[i])

0/3000
100/3000
200/3000
300/3000
400/3000
500/3000
600/3000
700/3000
800/3000
900/3000
1000/3000
1100/3000
1200/3000
1300/3000
1400/3000
1500/3000
1600/3000
1700/3000
1800/3000
1900/3000
2000/3000
2100/3000
2200/3000
2300/3000
2400/3000
2500/3000
2600/3000
2700/3000
2800/3000
2900/3000
Classes: 

class_3
class_2
class_4
class_1
class_5
class_0
class_6
class_7
class_8
class_9

A few example files: 

1_theo_42.png
1_theo_4.png
1_theo_34.png
1_theo_36.png
1_theo_49.png
1_theo_46.png
1_theo_43.png
1_theo_37.png
1_yweweler_12.png
1_yweweler_19.png


In [ ]:
class Trainer():

    def __init__(self, name):
        self.name = name

    def build_model(self, img_height, img_width, n_channels, n_classes):
        self.model = tf.keras.models.Sequential()
        self.model.add(tf.keras.layers.Input(shape=(img_height, img_width, n_channels)))
        self.model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Flatten())
        self.model.add(tf.keras.layers.Dense(256, activation='relu'))
        self.model.add(tf.keras.layers.BatchNormalization())
        self.model.add(tf.keras.layers.Dropout(0.5))
        self.model.add(tf.keras.layers.Dense(n_classes, activation='softmax'))

        # Compile model
        self.model.compile(
            loss='sparse_categorical_crossentropy',
            optimizer=tf.keras.optimizers.RMSprop(),
            metrics=['accuracy'],
        )

    def train_model(self, train_dataset, val_dataset, epochs):
        self.history = self.model.fit(train_dataset, epochs=epochs, validation_data=val_dataset)

    def plot_loss(self):
        # Plot the loss curves for training and validation.
        history_dict = self.history.history
        loss_values = history_dict['loss']
        val_loss_values = history_dict['val_loss']
        epochs = range(1, len(loss_values)+1)

        plt.figure(figsize=(8,6))
        plt.plot(epochs, loss_values, 'bo', label='Training loss')
        plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
        plt.title('Training and validation loss')
        plt.xlabel('Epochs')
        plt.ylabel('Loss')
        plt.legend()
        plt.show()

    def plot_accuracy(self):
        # Plot the accuracy curves for training and validation.
        history_dict = self.history.history
        acc_values = history_dict['accuracy']
        val_acc_values = history_dict['val_accuracy']
        epochs = range(1, len(acc_values)+1)

        plt.figure(figsize=(8,6))
        plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
        plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
        plt.title('Training and validation accuracy')
        plt.xlabel('Epochs')
        plt.ylabel('Accuracy')
        plt.legend()
        plt.show()

In [ ]:
# Declare constants
IMAGE_HEIGHT = 256
IMAGE_WIDTH = 256
BATCH_SIZE = 32
N_CHANNELS = 3
N_CLASSES = 10

# Make a dataset containing the training spectrograms
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="training",
                                             seed=0)

# Make a dataset containing the validation spectrogram
valid_dataset = tf.keras.preprocessing.image_dataset_from_directory(
                                             batch_size=BATCH_SIZE,
                                             validation_split=0.2,
                                             directory=os.path.join(OUTPUT_DIR, 'audio-images'),
                                             shuffle=True,
                                             color_mode='rgb',
                                             image_size=(IMAGE_HEIGHT, IMAGE_WIDTH),
                                             subset="validation",
                                             seed=0)

Found 3000 files belonging to 10 classes.
Using 2400 files for training.
Found 3000 files belonging to 10 classes.
Using 600 files for validation.


In [ ]:
# Function to prepare our datasets for modelling
def prepare(ds, augment=False):
    # Define our one transformation
    rescale = tf.keras.Sequential([tf.keras.layers.experimental.preprocessing.Rescaling(1./255)])
    flip_and_rotate = tf.keras.Sequential([
        tf.keras.layers.experimental.preprocessing.RandomFlip("horizontal_and_vertical"),
        tf.keras.layers.experimental.preprocessing.RandomRotation(0.2)
    ])
    
    # Apply rescale to both datasets and augmentation only to training
    ds = ds.map(lambda x, y: (rescale(x, training=True), y))
    if augment: ds = ds.map(lambda x, y: (flip_and_rotate(x, training=True), y))
    return ds

train_dataset = prepare(train_dataset, augment=False)
valid_dataset = prepare(valid_dataset, augment=False)

In [ ]:
model = tf.keras.models.Sequential()
model.add(tf.keras.layers.Input(shape=(IMAGE_HEIGHT, IMAGE_WIDTH, N_CHANNELS)))
model.add(tf.keras.layers.Conv2D(32, 3, strides=2, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Flatten())
model.add(tf.keras.layers.Dense(256, activation='relu'))
model.add(tf.keras.layers.BatchNormalization())
model.add(tf.keras.layers.Dropout(0.5))
model.add(tf.keras.layers.Dense(N_CLASSES, activation='softmax'))

# Compile model
model.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.RMSprop(),
    metrics=['accuracy'],
)

In [ ]:
# Train model for 10 epochs, capture the history
history = model.fit(train_dataset, epochs=10, validation_data=valid_dataset)

Epoch 1/10
75/75 [==============================] - ETA: 0s - loss: 0.1878 - accuracy: 0.9508

InvalidArgumentError: ignored

In [ ]:
# Plot the loss curves for training and validation.
history_dict = history.history
loss_values = history_dict['loss']
val_loss_values = history_dict['val_loss']
epochs = range(1, len(loss_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, loss_values, 'bo', label='Training loss')
plt.plot(epochs, val_loss_values, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()

In [ ]:
# Plot the accuracy curves for training and validation.
acc_values = history_dict['accuracy']
val_acc_values = history_dict['val_accuracy']
epochs = range(1, len(acc_values)+1)

plt.figure(figsize=(8,6))
plt.plot(epochs, acc_values, 'bo', label='Training accuracy')
plt.plot(epochs, val_acc_values, 'b', label='Validation accuracy')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
# Compute the final loss and accuracy
final_loss, final_acc = model.evaluate(valid_dataset, verbose=0)
print("Final loss: {0:.6f}, final accuracy: {1:.6f}".format(final_loss, final_acc))